# Tools
Tools extend what agents can do—letting them fetch real-time data, execute code, query external databases, and take actions in the world.

Basically: Can fetch data from any data source either PDFs , webBased data etc 

Demonstration of using webBased Tool , inbuild langchain tools

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [17]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma 
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

webBasedLoader=WebBaseLoader("https://vajiramandravi.com/current-affairs/seismic-waves/")
docs=webBasedLoader.load()

In [9]:
document_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=document_splitter.split_documents(docs)

In [10]:
db=Chroma.from_documents(documents,OpenAIEmbeddings())

/Users/apple/Desktop/GenAI/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [12]:
retriever=db.as_retriever()

# Create Retrival Tool 
The create_retriever_tool function in LangChain creates a specialized callable tool that allows a Language Model (LLM), typically within an autonomous agent, to actively search and retrieve relevant external information. 


In [41]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool=create_retriever_tool(retriever,"Sesmic_Waves_Tool" ,"Tool for answering questions about sesmic waves ")

Using Archix tool. from langhcina

In [16]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=api_wrapper)

In [42]:
tools=[retrieval_tool, wiki_tool, arxiv_tool]

# Agents
Intelligent systems that use a Large Language Model (LLM) to decide the control flow of an application ( Tools), reason about tasks, and use tools to achieve a goal . 

In [29]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")


In [36]:
## Using LangchainHub for premade prompt templates
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")  # Changed from openai-functions-agent

In [43]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, tools, prompt)

In [44]:
# Agent Executor 
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent, tools=tools, verbose=True)

In [46]:
agent_executor.invoke( {"input":" What is wikipeida?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Wikipedia`


Page: Wikipedia
Summary: Wikipedia is a free online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and the wiki software MediaWiki. Wikipedia is a free online encyclopedia that is written and maintained by a community of volunteers called Wikipedians. It operates through open collaboration using the wiki software known as MediaWiki.

> Finished chain.


{'input': ' What is wikipeida?',
 'output': 'Wikipedia is a free online encyclopedia that is written and maintained by a community of volunteers called Wikipedians. It operates through open collaboration using the wiki software known as MediaWiki.'}